# Document Data Modeling Using List Order

When should you order data in Aerospike?

Given that each record in Aerospike adds a significant number of bytes to the index, the intuition is to store maps and lists of data – documents of data – rather than individual values. 

In particular, a list represents a lightweight record structure that optimizes storage, and Aerospike's List Operations provide an intuitive order to document-oriented data. This notebook shares how Aerospike facilitates working with list data, covering the following topics:
* Ordering
* Value Comparisons
* Intervals
* Relative Range Rank

## Notebook Setup

### Import Jupyter Java Integration 

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;

IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Start Aerospike

In [2]:
%sh asd

### Download the Aerospike Java Client

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.0.0</version>
  </dependency>
</dependencies>

### Start the Aerospike Java Client and Connect

The default cluster location for the Docker container is *localhost* port *3000*. If your cluster is not running on your local machine, modify *localhost* and *3000* to the values for your Aerospike cluster.

In [4]:
import com.aerospike.client.AerospikeClient;

AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Initialized the client and connected to the cluster.


# Aerospike Server Optionally Maintains Data in Order

When working with real time data at scale, performance is everything and each operation has a performance cost. For List operations, Aerospike provides performance analysis for [all operations](https://www.aerospike.com/docs/guide/cdt-list-performance.html). Because Aerospike was made for real-time data processing of extremely large datasets, analysis is provided in [Big O notation]( https://en.wikipedia.org/wiki/Big_O_notation). 

Storing ordered data has its cost. 

* An application that has many small sparse data reads will commonly benefit little from using server cycles to order data.
* An application that has fewer larger data reads.  


## Unordered Data – Putting Data into Aerospike Preserves Order

Putting a list or map into Aerospike as **unordered** preserves the original order. This order can be accessed by **index** operations, and is maintained by adding new data using the **append** operation. Using the **Insert** operation to add at an index, inserts before a preexisting item, and increments the index for the preexisting item and subsequent items.

For more information on [list operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html).

For more information on [map operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapOperation.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapOperation.html).

## Ordered Data – Order a List or Map Upon Creation

The best practice for ordering a bin is to order it upon creation in Aerospike. When there are few or no items in the list, the computational cost of ordering the bin is smallest. 

Ordering can be done at any time using the **setOrder** list operation. 

Use append to add to an ordered list. Appending will reorder the list, as necessary.

### Create the List `[1]` 

In [5]:
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;
import java.util.Map;

ArrayList<Integer> origListInt = new ArrayList<Integer>();
origListInt.add(1);

System.out.println("Original Integer List: " + origListInt);

Original Integer List: [1]


### Put The List Into Two Aerospike Bins

In [6]:
import com.aerospike.client.Key;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;

Integer theKey = 0;
String listSet = "orderset1";
String listNamespace = "test";
String listOrderedBin = "ordered";
String listUnorderedBin = "unordered";
ClientPolicy clientPolicy = new ClientPolicy();

Key key = new Key(listNamespace, listSet, theKey);
Bin bin1 = new Bin(listOrderedBin, origListInt);
Bin bin2 = new Bin(listUnorderedBin, origListInt);

client.put(clientPolicy.writePolicyDefault, key, bin1, bin2);
System.out.println("Put list into record with bins: " + listOrderedBin + " and " 
                    + listUnorderedBin);

Put list into record with bins: ordered and unordered


### Set the Sort Order for the Ordered Bin

The [default order](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOrder.html) applied to a bin is unordered.

In [7]:
import com.aerospike.client.Record;
import com.aerospike.client.Operation;
import com.aerospike.client.Value;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListOrder;

Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
                        ListOperation.setOrder(listOrderedBin, ListOrder.ORDERED)
                        );

System.out.println("Configured ordering policy to bin: " + listOrderedBin);

Configured ordering policy to bin: ordered


### Append Elements to Both Bins

Append additional elements `[4, 7, 3, 9, 26, 11]`

In [8]:
ArrayList<Value> addListInt = new ArrayList<Value>();
addListInt.add(Value.get(4));
addListInt.add(Value.get(7));
addListInt.add(Value.get(3));
addListInt.add(Value.get(9));
addListInt.add(Value.get(26));
addListInt.add(Value.get(11));

System.out.println("Appending the following list items to both lists: " + addListInt);

Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
                        ListOperation.appendItems(listOrderedBin, addListInt),
                        ListOperation.appendItems(listUnorderedBin, addListInt) 
                        );
                        
Record finalListComp = client.get(null, key);
System.out.println("The Unordered List is " + finalListComp.getValue(listUnorderedBin));
System.out.println("The Ordered List is " + finalListComp.getValue(listOrderedBin));

Appending the following list items to both lists: [4, 7, 3, 9, 26, 11]
The Unordered List is [1, 4, 7, 3, 9, 26, 11]
The Ordered List is [1, 3, 4, 7, 9, 11, 26]


# Aerospike Clients Provide JIT Ordering

Aerospike list operations access data in lists by index or rank order. Index operations allow access by order of insertion. Aerospike client sorts data just-in-time for rank operations.

## Rank Operations Sort All Data Types

Rank operations order and return data from Aerospike. Rather than just order individual data types, Aerospike client manages a comprehensive order of all data types: 

1.	NIL
2.	Boolean (Reserved for the future.)
3.	Integer
4.	String
5.	List
6.	Map
7.	Bytes
8. 	Double
9.	GeoJSON
10.	INF

Elements of type are ordered by type. For more information on the order of [data types](https://aerospike.com/docs/guide/cdt-ordering.html), go [here](https://aerospike.com/docs/guide/cdt-ordering.html).

Here's how you use them.

### Using Rank Operations on Lists

#### Create A Mixed Data Type List

Create a list containing letters, integers, and a nessted list:

`['c', 3, 26, [25, 'y', 1], 11, 9, 4, 'a', 1, 7]`

In [9]:
ArrayList<Value> subMixedList = new ArrayList<Value>();
subMixedList.add(Value.get(25));
subMixedList.add(Value.get("y"));
subMixedList.add(Value.get(1));

System.out.println("Sub Mixed Type List: " + subMixedList);

ArrayList<Value> mixedDataList = new ArrayList<Value>();
mixedDataList.add(Value.get("c"));
mixedDataList.add(Value.get(3));
mixedDataList.add(Value.get(26));
mixedDataList.add(Value.get(subMixedList));
mixedDataList.add(Value.get(11));
mixedDataList.add(Value.get(9));
mixedDataList.add(Value.get(4));
mixedDataList.add(Value.get("a"));
mixedDataList.add(Value.get(1));
mixedDataList.add(Value.get(7));

System.out.println("Mixed Type Data List: " + mixedDataList);

Sub Mixed Type List: [25, y, 1]
Mixed Type Data List: [c, 3, 26, [25, y, 1], 11, 9, 4, a, 1, 7]


#### Put the List into Two Aerospike Bins

In [10]:
Integer mixedKey = 1;
String listSet = "orderset1";

Key key = new Key(listNamespace, listSet, mixedKey);
Bin bin1 = new Bin(listOrderedBin, mixedDataList);
Bin bin2 = new Bin(listUnorderedBin, mixedDataList);

client.put(clientPolicy.writePolicyDefault, key, bin1, bin2);
System.out.println("Created record with bins: " + listOrderedBin + " and " 
                    + listUnorderedBin);

Created record with bins: ordered and unordered


#### Configure Sort Order for the Ordered Bin

In [11]:
Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
    ListOperation.setOrder(listOrderedBin, ListOrder.ORDERED)
    );
Record finalListComp = client.get(null, key);

System.out.println("Configured ordering policy to bin: " + listOrderedBin);
System.out.println("The Unordered List is " + finalListComp.getValue(listUnorderedBin));
System.out.println("The Ordered List is " + finalListComp.getValue(listOrderedBin));

Configured ordering policy to bin: ordered
The Unordered List is [c, 3, 26, [25, y, 1], 11, 9, 4, a, 1, 7]
The Ordered List is [1, 3, 4, 7, 9, 11, 26, a, c, [25, y, 1]]


#### What is the range of elements between `NIL` and `10`?

When comparing ranges in Aerospike, the lower end is *inclusive*, and the upper end is *exclusive*

The singleton `NIL` is accessed in Java as `Value.NULL`. For more information on [Values](https://www.aerospike.com/apidocs/java/com/aerospike/client/Value.ListValue.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/Value.ListValue.html).

In [12]:
import com.aerospike.client.cdt.ListReturnType;

Value lowEnd = Value.NULL;
Value highEnd = Value.get(10);

Record rangeNIL10 = client.get(null, key);
rangeNIL10 = client.operate(client.writePolicyDefault, key, 
                ListOperation.getByValueRange(listOrderedBin, lowEnd, highEnd, 
                                              ListReturnType.VALUE),
                ListOperation.getByValueRange(listUnorderedBin, lowEnd, highEnd, 
                                              ListReturnType.VALUE)
                );

System.out.println("The Unordered List range is " + rangeNIL10.getValue(listUnorderedBin));
System.out.println("The Ordered List range is " + rangeNIL10.getValue(listOrderedBin));

The Unordered List range is [3, 9, 4, 1, 7]
The Ordered List range is [1, 3, 4, 7, 9]


#### What is the range of elements between `'a'` and `[ ]`?

In [13]:
ArrayList<Value> emptyList = new ArrayList<Value>();

Value lowEnd = Value.get("a");
Value highEnd = Value.get(emptyList);

Record rangeANullList = client.get(null, key);
rangeANullList = client.operate(client.writePolicyDefault, key, 
                ListOperation.getByValueRange(listOrderedBin, lowEnd, highEnd, 
                                              ListReturnType.VALUE),
                ListOperation.getByValueRange(listUnorderedBin, lowEnd, highEnd, 
                                              ListReturnType.VALUE)
                );

System.out.println("The Unordered List range is " + rangeANullList.getValue(listUnorderedBin));
System.out.println("The Ordered List range is " + rangeANullList.getValue(listOrderedBin));

The Unordered List range is [c, a]
The Ordered List range is [a, c]


#### What is the range of elements between `[ ]` and `INF`?

The singleton `INF` is accessed in Java as `Value.INFINITY`. For more information on [Values](https://www.aerospike.com/apidocs/java/com/aerospike/client/Value.ListValue.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/Value.ListValue.html).

In [14]:
Value lowEnd = Value.get(emptyList);
Value highEnd = Value.INFINITY;

Record rangeNullInf = client.get(null, key);
rangeNullInf = client.operate(client.writePolicyDefault, key, 
                ListOperation.getByValueRange(listOrderedBin, lowEnd, highEnd, 
                                              ListReturnType.VALUE),
                ListOperation.getByValueRange(listUnorderedBin, lowEnd, highEnd, 
                                              ListReturnType.VALUE)
                );

System.out.println("The Unordered List range is " + rangeNullInf.getValue(listUnorderedBin));
System.out.println("The Ordered List range is " + rangeNullInf.getValue(listOrderedBin));

The Unordered List range is [[25, y, 1]]
The Ordered List range is [[25, y, 1]]


### Rank Works Reliably Independent of List Ordering

As demonstrated above, rank works identically, no matter whether the list is ordered or not.

### Using Rank on Lists of Tuples

Ordering lists containing sublists and maps works intuitively and differently from simple lists.

1. Sublists or maps in an ordered list are not implicitly ordered.
2. Items are inserted in rank order. 
3. Greater List length results in higher rank 

Here are some examples.

#### Create a Sorted and Unsorted Empty List

In [15]:
Integer listOfListsKey = 2;
String listSet = "orderset1";

Key key = new Key(listNamespace, listSet, listOfListsKey);
Bin bin1 = new Bin(listOrderedBin, emptyList);
Bin bin2 = new Bin(listUnorderedBin, emptyList);

client.put(clientPolicy.writePolicyDefault, key, bin1, bin2);
System.out.println("Created record with bins: " + listOrderedBin + " and " 
                    + listUnorderedBin);
                    
Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
    ListOperation.setOrder(listOrderedBin, ListOrder.ORDERED)
    );
Record initialEmptyLists = client.get(null, key);

System.out.println("Configured ordering policy to bin: " + listOrderedBin);
System.out.println("The Unordered List is " + initialEmptyLists.getValue(listUnorderedBin));
System.out.println("The Ordered List is " + initialEmptyLists.getValue(listOrderedBin));

Created record with bins: ordered and unordered
Configured ordering policy to bin: ordered
The Unordered List is []
The Ordered List is []


#### Add  `[1, 1]`

In [16]:
ArrayList<Integer> thisTuple = new ArrayList<Integer>();
thisTuple.add(1);
thisTuple.add(1);

Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
    ListOperation.append(listOrderedBin, Value.get(thisTuple)),
    ListOperation.append(listUnorderedBin, Value.get(thisTuple))    
    );
Record listWith11 = client.get(null, key);

System.out.println("The Unordered List is " + listWith11.getValue(listUnorderedBin));
System.out.println("The Ordered List is " + listWith11.getValue(listOrderedBin));

The Unordered List is [[1, 1]]
The Ordered List is [[1, 1]]


#### Add  `[1, 3]`

In [17]:
ArrayList<Integer> thisTuple = new ArrayList<Integer>();
thisTuple.add(1);
thisTuple.add(3);

Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
    ListOperation.append(listOrderedBin, Value.get(thisTuple)),
    ListOperation.append(listUnorderedBin, Value.get(thisTuple))    
    );
Record listWith13 = client.get(null, key);

System.out.println("The Unordered List is " + listWith13.getValue(listUnorderedBin));
System.out.println("The Ordered List is " + listWith13.getValue(listOrderedBin));

The Unordered List is [[1, 1], [1, 3]]
The Ordered List is [[1, 1], [1, 3]]


#### Add  `[1, 2, 0, 1]`

In [18]:
ArrayList<Integer> thisTuple = new ArrayList<Integer>();
thisTuple.add(1);
thisTuple.add(2);
thisTuple.add(0);
thisTuple.add(1);

Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
    ListOperation.append(listOrderedBin, Value.get(thisTuple)),
    ListOperation.append(listUnorderedBin, Value.get(thisTuple))    
    );
Record listWith1201 = client.get(null, key);

System.out.println("The Unordered List is " + listWith1201.getValue(listUnorderedBin));
System.out.println("The Ordered List is " + listWith1201.getValue(listOrderedBin));

The Unordered List is [[1, 1], [1, 3], [1, 2, 0, 1]]
The Ordered List is [[1, 1], [1, 2, 0, 1], [1, 3]]


#### Add  `[1, 2]`

In [19]:
ArrayList<Integer> thisTuple = new ArrayList<Integer>();
thisTuple.add(1);
thisTuple.add(2);

Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
    ListOperation.append(listOrderedBin, Value.get(thisTuple)),
    ListOperation.append(listUnorderedBin, Value.get(thisTuple))    
    );
Record listWith12 = client.get(null, key);

System.out.println("The Unordered List is " + listWith12.getValue(listUnorderedBin));
System.out.println("The Ordered List is " + listWith12.getValue(listOrderedBin));

The Unordered List is [[1, 1], [1, 3], [1, 2, 0, 1], [1, 2]]
The Ordered List is [[1, 1], [1, 2], [1, 2, 0, 1], [1, 3]]


### Using Intervals 

Interval operations deliver parts of a JIT-ordered list, whether or not the data is sorted. Intervals use the same consistent sort order across data types as rank:
* `[2, NIL]` is to the right of anything `[1, ...]`, regardless of element count.
* `[1, INF]` is to the left of anything `[1, ...]` and to the right of anything `[2, ...]`.

The value is in using lists as a lightweight record structure, which can be queried by interval.

Here are some examples.

#### Create a List of Tuples
`[[2, 7], [3, 1], [1, 1], [2, 5], [1, 2], [0, 3]]`

In [20]:
ArrayList<Integer> subTuple0 = new ArrayList<Integer>();
subTuple0.add(2);
subTuple0.add(7);

ArrayList<Integer> subTuple1 = new ArrayList<Integer>();
subTuple1.add(3);
subTuple1.add(1);

ArrayList<Integer> subTuple2 = new ArrayList<Integer>();
subTuple2.add(1);
subTuple2.add(1);

ArrayList<Integer> subTuple3 = new ArrayList<Integer>();
subTuple3.add(2);
subTuple3.add(5);

ArrayList<Integer> subTuple4 = new ArrayList<Integer>();
subTuple4.add(1);
subTuple4.add(2);

ArrayList<Integer> subTuple5 = new ArrayList<Integer>();
subTuple5.add(0);
subTuple5.add(3);

ArrayList<Value> tupleList = new ArrayList<Value>();
tupleList.add(Value.get(subTuple0));
tupleList.add(Value.get(subTuple1));
tupleList.add(Value.get(subTuple2));
tupleList.add(Value.get(subTuple3));
tupleList.add(Value.get(subTuple4));
tupleList.add(Value.get(subTuple5));

System.out.println("Tuple List: " + tupleList);

Tuple List: [[2, 7], [3, 1], [1, 1], [2, 5], [1, 2], [0, 3]]


#### Put the List Into Two Aerospike Bins

In [21]:
Integer tupleKey = 3;
String listSet = "orderset1";

Key key = new Key(listNamespace, listSet, tupleKey);
Bin bin1 = new Bin(listOrderedBin, tupleList);
Bin bin2 = new Bin(listUnorderedBin, tupleList);

client.put(clientPolicy.writePolicyDefault, key, bin1, bin2);
System.out.println("Created record with bins: " + listOrderedBin + " and " 
                    + listUnorderedBin);

Created record with bins: ordered and unordered


#### Sort the Ordered Bin

In [22]:
Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
    ListOperation.setOrder(listOrderedBin, ListOrder.ORDERED)
    );
Record initialEmptyLists = client.get(null, key);

System.out.println("Sorted the bin: " + listOrderedBin);

Sorted the bin: ordered


#### Get the Interval Between `[1, NIL]` and `[2, NIL]`

In [23]:
ArrayList<Value> lowTuple = new ArrayList<Value>();
lowTuple.add(Value.get(1));
lowTuple.add(Value.NULL);

ArrayList<Value> highTuple = new ArrayList<Value>();
highTuple.add(Value.get(2));
highTuple.add(Value.NULL);

Record between1and2 = client.get(null, key);
between1and2 = client.operate(client.writePolicyDefault, key, 
    ListOperation.getByValueRange(listOrderedBin, Value.get(lowTuple), Value.get(highTuple), 
        ListReturnType.VALUE),
    ListOperation.getByValueRange(listUnorderedBin, Value.get(lowTuple), Value.get(highTuple), 
        ListReturnType.VALUE)    
    );

System.out.println("The Unordered List between " + lowTuple + " and " + highTuple + " is " 
    + between1and2.getValue(listUnorderedBin));
System.out.println("The Ordered List between " + lowTuple + " and " + highTuple + " is " 
    + between1and2.getValue(listOrderedBin));

The Unordered List between [1, null] and [2, null] is [[1, 1], [1, 2]]
The Ordered List between [1, null] and [2, null] is [[1, 1], [1, 2]]


#### Get the Interval Between `[1, NIL]` and `[1, INF]`.

In [24]:
ArrayList<Value> lowTuple = new ArrayList<Value>();
lowTuple.add(Value.get(1));
lowTuple.add(Value.NULL);

ArrayList<Value> highTuple = new ArrayList<Value>();
highTuple.add(Value.get(1));
highTuple.add(Value.INFINITY);

Record between1NILand1INF = client.get(null, key);
between1NILand1INF = client.operate(client.writePolicyDefault, key, 
    ListOperation.getByValueRange(listOrderedBin, Value.get(lowTuple), Value.get(highTuple), 
        ListReturnType.VALUE),
    ListOperation.getByValueRange(listUnorderedBin, Value.get(lowTuple), Value.get(highTuple), 
        ListReturnType.VALUE)    
    );

System.out.println("The Unordered List between " + lowTuple + " and " + highTuple + " is " 
    + between1NILand1INF.getValue(listUnorderedBin));
System.out.println("The Ordered List between " + lowTuple + " and " + highTuple + " is " 
    + between1NILand1INF.getValue(listOrderedBin));

The Unordered List between [1, null] and [1, INF] is [[1, 1], [1, 2]]
The Ordered List between [1, null] and [1, INF] is [[1, 1], [1, 2]]


### The Ordering of the Data is Independent of Results 

As demonstrated, the results are the same whether the data is ordered or unordered in the database.

# Examples

## Example 1 – IoT Model

Modeling with tuples is powerful. Consider a smart thermometer taking a reading one time per minute. A clean model for storage is a list of tuples.

For more information on this [IoT example](https://dev.to/aerospike/aerospike-modeling-iot-sensors-453a), go [here](https://dev.to/aerospike/aerospike-modeling-iot-sensors-453a).

### Create Sample Data

Record Format – `[minutes-since-epoch, temperature]
[14062, 39.04]
[14063, 39.78]
[14065, 40.07]
[14064, 40.89]
[14066, 41.18]
[14067, 40.93]`

In [25]:
ArrayList<Value> subTuple0 = new ArrayList<Value>();
subTuple0.add(Value.get(14062));
subTuple0.add(Value.get(39.04));

ArrayList<Value> subTuple1 = new ArrayList<Value>();
subTuple1.add(Value.get(14063));
subTuple1.add(Value.get(39.78));

ArrayList<Value> subTuple2 = new ArrayList<Value>();
subTuple2.add(Value.get(14065));
subTuple2.add(Value.get(40.07));

ArrayList<Value> subTuple3 = new ArrayList<Value>();
subTuple3.add(Value.get(14064));
subTuple3.add(Value.get(40.89));

ArrayList<Value> subTuple4 = new ArrayList<Value>();
subTuple4.add(Value.get(14066));
subTuple4.add(Value.get(41.18));

ArrayList<Value> subTuple5 = new ArrayList<Value>();
subTuple5.add(Value.get(14067));
subTuple5.add(Value.get(40.93));

ArrayList<Value> tupleListIoT = new ArrayList<Value>();
tupleListIoT.add(Value.get(subTuple0));
tupleListIoT.add(Value.get(subTuple1));
tupleListIoT.add(Value.get(subTuple2));
tupleListIoT.add(Value.get(subTuple3));
tupleListIoT.add(Value.get(subTuple4));
tupleListIoT.add(Value.get(subTuple5));

System.out.println("IoT Tuple List: " + tupleListIoT);

IoT Tuple List: [[14062, 39.04], [14063, 39.78], [14065, 40.07], [14064, 40.89], [14066, 41.18], [14067, 40.93]]


### Put the List in an Unordered Bin

In [26]:
Integer keyIoT = 4;
String listSet = "orderset1";
ClientPolicy clientPolicy = new ClientPolicy();

Key key = new Key(listNamespace, listSet, keyIoT);
Bin bin1 = new Bin(listUnorderedBin, tupleListIoT);

client.put(clientPolicy.writePolicyDefault, key, bin1);
System.out.println("Created record with bin: " + listUnorderedBin);

Created record with bin: unordered


### Get Temperature Readings Between Minute 14063 and 14064

In [27]:
ArrayList<Value> lowTuple = new ArrayList<Value>();
lowTuple.add(Value.get(14063));
lowTuple.add(Value.NULL);

ArrayList<Value> highTuple = new ArrayList<Value>();
highTuple.add(Value.get(14064));
highTuple.add(Value.INFINITY);

Record between14063and14064 = client.get(null, key);
between14063and14064 = client.operate(client.writePolicyDefault, key, 
    ListOperation.getByValueRange(listUnorderedBin, Value.get(lowTuple), Value.get(highTuple), 
        ListReturnType.VALUE)    
    );

System.out.println("The readings between " + lowTuple + " and " + highTuple + " are " 
    + between14063and14064.getValue(listUnorderedBin));

The readings between [14063, null] and [14064, INF] are [[14063, 39.78], [14064, 40.89]]


## Example 2 – 100 Meter Sprint Leaderboard

Here is an example with mixed type data and overlapping datasets. It also highlights Aerospike's ability to find the tuples closest to a given value and **relative range ranking**.

### Create Overlapping Mixed-Type Data Sample

Record format – `[Time, Athlete, Location, Date]`

#### Create Example Data Set 1

wr100m1 – 
`[
[10.06, "Bob Hayes", "Tokyo, Japan", "October 15, 1964"],
[10.03, "Jim Hines", "Sacramento, USA", "June 20, 1968"],
[10.02, "Charles Greene", "Mexico City, Mexico", "October 13, 1968"],
[9.95, "Jim Hines", "Mexico City, Mexico", "October 14, 1968"],
[9.93, "Calvin Smith", "Colorado Springs, USA", "July 3, 1983"],
[9.93, "Carl Lewis", "Rome, Italy", "August 30, 1987"],
[9.92, "Carl Lewis", "Seoul, South Korea", "September 24, 1988"]
]`

In [28]:
ArrayList<Value> wr100One0 = new ArrayList<Value>();
wr100One0.add(Value.get(10.06));
wr100One0.add(Value.get("Bob Hayes"));
wr100One0.add(Value.get("Tokyo, Japan"));
wr100One0.add(Value.get("October 15, 1964"));

ArrayList<Value> wr100One1 = new ArrayList<Value>();
wr100One1.add(Value.get(10.03));
wr100One1.add(Value.get("Jim Hines"));
wr100One1.add(Value.get("Sacramento, USA"));
wr100One1.add(Value.get("June 20,1968"));

ArrayList<Value> wr100One2 = new ArrayList<Value>();
wr100One2.add(Value.get(10.02));
wr100One2.add(Value.get("Charles Greene"));
wr100One2.add(Value.get("Mexico City, Mexico"));
wr100One2.add(Value.get("October 13, 1968"));

ArrayList<Value> wr100One3 = new ArrayList<Value>();
wr100One3.add(Value.get(9.95));
wr100One3.add(Value.get("Jim Hines"));
wr100One3.add(Value.get("Mexico City, Mexico"));
wr100One3.add(Value.get("October 14, 1968"));

ArrayList<Value> wr100One4 = new ArrayList<Value>();
wr100One4.add(Value.get(9.93));
wr100One4.add(Value.get("Calvin Smith"));
wr100One4.add(Value.get("Colorado Springs, USA"));
wr100One4.add(Value.get("July 3, 1983"));

ArrayList<Value> wr100One5 = new ArrayList<Value>();
wr100One5.add(Value.get(9.93));
wr100One5.add(Value.get("Carl Lewis"));
wr100One5.add(Value.get("Rome, Italy"));
wr100One5.add(Value.get("August 30, 1987"));

ArrayList<Value> wr100One6 = new ArrayList<Value>();
wr100One6.add(Value.get(9.92));
wr100One6.add(Value.get("Carl Lewis"));
wr100One6.add(Value.get("Seoul, South Korea"));
wr100One6.add(Value.get("September 24, 1988"));


ArrayList<Value> wr100m1List = new ArrayList<Value>();
wr100m1List.add(Value.get(wr100One0));
wr100m1List.add(Value.get(wr100One1));
wr100m1List.add(Value.get(wr100One2));
wr100m1List.add(Value.get(wr100One3));
wr100m1List.add(Value.get(wr100One4));
wr100m1List.add(Value.get(wr100One5));
wr100m1List.add(Value.get(wr100One6));


System.out.println("100m World Record Holder List 1: " + wr100m1List);

100m World Record Holder List 1: [[10.06, Bob Hayes, Tokyo, Japan, October 15, 1964], [10.03, Jim Hines, Sacramento, USA, June 20,1968], [10.02, Charles Greene, Mexico City, Mexico, October 13, 1968], [9.95, Jim Hines, Mexico City, Mexico, October 14, 1968], [9.93, Calvin Smith, Colorado Springs, USA, July 3, 1983], [9.93, Carl Lewis, Rome, Italy, August 30, 1987], [9.92, Carl Lewis, Seoul, South Korea, September 24, 1988]]


#### Create Example Data Set 2

wr_100m_2 – `[
[9.93, "Carl Lewis", "Rome, Italy", "August 30, 1987"],
[9.92, "Carl Lewis", "Seoul, South Korea", "September 24, 1988"],
[9.90, "Leroy Burrell", "New York, USA", "June 14, 1991"],
[9.86, "Carl Lewis", "Tokyo, Japan", "August 25, 1991"],
[9.85, "Leroy Burrell", "Lausanne, Switzerland", "July 6, 1994"],
[9.84, "Donovan Bailey", "Atlanta, USA", "July 27, 1996"],
[9.79, "Maurice Greene", "Athens, Greece", "June 16, 1999"],
[9.77, "Asafa Powell", "Athens, Greece", "June 14, 2005"],
[9.77, "Asafa Powell", "Gateshead, England", "June 11, 2006"],
[9.77, "Asafa Powell", "Zurich, Switzerland", "August 18, 2006"],
[9.74, "Asafa Powell", "Rieti, Italy", "September 9, 2007"],
[9.72, "Usain Bolt", "New York, USA", "May 31, 2008"],
[9.69, "Usain Bolt", "Beijing, China", "August 16, 2008"],
[9.58, "Usain Bolt", "Berlin, Germany", "August 16, 2009"]
]`

In [29]:
ArrayList<Value> wr100Two0 = new ArrayList<Value>();
wr100Two0.add(Value.get(9.93));
wr100Two0.add(Value.get("Carl Lewis"));
wr100Two0.add(Value.get("Rome, Italy"));
wr100Two0.add(Value.get("August 30, 1987"));

ArrayList<Value> wr100Two1 = new ArrayList<Value>();
wr100Two1.add(Value.get(9.92));
wr100Two1.add(Value.get("Carl Lewis"));
wr100Two1.add(Value.get("Seoul, South Korea"));
wr100Two1.add(Value.get("September 24, 1988"));

ArrayList<Value> wr100Two2 = new ArrayList<Value>();
wr100Two2.add(Value.get(9.90));
wr100Two2.add(Value.get("Leroy Burrell"));
wr100Two2.add(Value.get("New York, USA"));
wr100Two2.add(Value.get("June 14, 1991"));

ArrayList<Value> wr100Two3 = new ArrayList<Value>();
wr100Two3.add(Value.get(9.86));
wr100Two3.add(Value.get("Carl Lewis"));
wr100Two3.add(Value.get("Tokyo, Japan"));
wr100Two3.add(Value.get("August 25, 1991"));

ArrayList<Value> wr100Two4 = new ArrayList<Value>();
wr100Two4.add(Value.get(9.85));
wr100Two4.add(Value.get("Leroy Burrell"));
wr100Two4.add(Value.get("Lausanne, Switzerland"));
wr100Two4.add(Value.get("July 6, 1994"));

ArrayList<Value> wr100Two5 = new ArrayList<Value>();
wr100Two5.add(Value.get(9.84));
wr100Two5.add(Value.get("Donovan Bailey"));
wr100Two5.add(Value.get("Atlanta, USA"));
wr100Two5.add(Value.get("July 27, 1996"));

ArrayList<Value> wr100Two6 = new ArrayList<Value>();
wr100Two6.add(Value.get(9.79));
wr100Two6.add(Value.get("Maurice Greene"));
wr100Two6.add(Value.get("Athens, Greece"));
wr100Two6.add(Value.get("June 16, 1999"));

ArrayList<Value> wr100Two7 = new ArrayList<Value>();
wr100Two7.add(Value.get(9.77));
wr100Two7.add(Value.get("Asafa Powell"));
wr100Two7.add(Value.get("Athens, Greece"));
wr100Two7.add(Value.get("June 14, 2005"));

ArrayList<Value> wr100Two8 = new ArrayList<Value>();
wr100Two8.add(Value.get(9.77));
wr100Two8.add(Value.get("Asafa Powell"));
wr100Two8.add(Value.get("Gateshead, England"));
wr100Two8.add(Value.get("June 11, 2006"));

ArrayList<Value> wr100Two9 = new ArrayList<Value>();
wr100Two9.add(Value.get(9.77));
wr100Two9.add(Value.get("Asafa Powell"));
wr100Two9.add(Value.get("Zurich, Switzerland"));
wr100Two9.add(Value.get("August 18, 2006"));

ArrayList<Value> wr100Two10 = new ArrayList<Value>();
wr100Two10.add(Value.get(9.74));
wr100Two10.add(Value.get("Asafa Powell"));
wr100Two10.add(Value.get("Rieti, Italy"));
wr100Two10.add(Value.get("September 9, 2007"));

ArrayList<Value> wr100Two11 = new ArrayList<Value>();
wr100Two11.add(Value.get(9.72));
wr100Two11.add(Value.get("Usain Bolt"));
wr100Two11.add(Value.get("New York, USA"));
wr100Two11.add(Value.get("May 31, 2008"));

ArrayList<Value> wr100Two12 = new ArrayList<Value>();
wr100Two12.add(Value.get(9.69));
wr100Two12.add(Value.get("Usain Bolt"));
wr100Two12.add(Value.get("Beijing, China"));
wr100Two12.add(Value.get("August 16, 2008"));

ArrayList<Value> wr100Two13 = new ArrayList<Value>();
wr100Two13.add(Value.get(9.58));
wr100Two13.add(Value.get("Usain Bolt"));
wr100Two13.add(Value.get("Berlin, Germany"));
wr100Two13.add(Value.get("August 16, 2009"));


ArrayList<Value> wr100m2List = new ArrayList<Value>();
wr100m2List.add(Value.get(wr100Two0));
wr100m2List.add(Value.get(wr100Two1));
wr100m2List.add(Value.get(wr100Two2));
wr100m2List.add(Value.get(wr100Two3));
wr100m2List.add(Value.get(wr100Two4));
wr100m2List.add(Value.get(wr100Two5));
wr100m2List.add(Value.get(wr100Two6));
wr100m2List.add(Value.get(wr100Two7));
wr100m2List.add(Value.get(wr100Two8));
wr100m2List.add(Value.get(wr100Two9));
wr100m2List.add(Value.get(wr100Two10));
wr100m2List.add(Value.get(wr100Two11));
wr100m2List.add(Value.get(wr100Two12));
wr100m2List.add(Value.get(wr100Two13));

System.out.println("100m World Record Holder List 2: " + wr100m2List);

100m World Record Holder List 2: [[9.93, Carl Lewis, Rome, Italy, August 30, 1987], [9.92, Carl Lewis, Seoul, South Korea, September 24, 1988], [9.9, Leroy Burrell, New York, USA, June 14, 1991], [9.86, Carl Lewis, Tokyo, Japan, August 25, 1991], [9.85, Leroy Burrell, Lausanne, Switzerland, July 6, 1994], [9.84, Donovan Bailey, Atlanta, USA, July 27, 1996], [9.79, Maurice Greene, Athens, Greece, June 16, 1999], [9.77, Asafa Powell, Athens, Greece, June 14, 2005], [9.77, Asafa Powell, Gateshead, England, June 11, 2006], [9.77, Asafa Powell, Zurich, Switzerland, August 18, 2006], [9.74, Asafa Powell, Rieti, Italy, September 9, 2007], [9.72, Usain Bolt, New York, USA, May 31, 2008], [9.69, Usain Bolt, Beijing, China, August 16, 2008], [9.58, Usain Bolt, Berlin, Germany, August 16, 2009]]


#### Put Data Set 1 Into An Aerospike Bin 

Data Set 2 will be used later.

In [30]:
Integer keyWR = 5;
String listSet = "orderset1";
String listLeadersBin = "leaders";

Key key = new Key(listNamespace, listSet, keyWR);
Bin bin1 = new Bin(listLeadersBin, wr100m1List);

client.put(clientPolicy.writePolicyDefault, key, bin1);
System.out.println("Created record with bin: " + listLeadersBin);

Created record with bin: leaders


#### Sort the Bin

In [31]:
Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
    ListOperation.setOrder(listLeadersBin, ListOrder.ORDERED)
    );

System.out.println("Ordered bin: " + listLeadersBin);

Ordered bin: leaders


### Find the Closest to 10.0 Seconds using Relative Range Rank

Sometimes the goal is to find the closes matches to a given data point. Aerospike provides this using **Relative Range Rank**. Finding the closest elements to 10 seconds is finding the closest relative range items to `[10.0, NIL]`.

In [32]:
ArrayList<Value> searchTuple = new ArrayList<Value>();
searchTuple.add(Value.get(10.0));
searchTuple.add(Value.NULL);

Integer searchRank = -1;
Integer searchQty = 2;

Record closestTo10 = client.get(null, key);
closestTo10 = client.operate(client.writePolicyDefault, key, 
    ListOperation.getByValueRelativeRankRange(listLeadersBin, Value.get(searchTuple), 
        searchRank, searchQty, ListReturnType.VALUE)
    );

System.out.println("The Closest World Records to 10.0 seconds are " + 
    closestTo10.getValue(listLeadersBin));

The Closest World Records to 10.0 seconds are [[9.95, Jim Hines, Mexico City, Mexico, October 14, 1968], [10.02, Charles Greene, Mexico City, Mexico, October 13, 1968]]


### Combine the Two Lists into One Ordered Unique List

#### Create A List Policy to Remove Duplicates

A **list write policy** contains the **sort order** and **flags** to govern the addition. For information on the [flags](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListWriteFlags.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListWriteFlags.html). 

Because the two data sets contain overlapping records, the operation needs to apply the following policies:
* ADD_UNIQUE – Add all unique items to the list.
* NO_FAIL – Duplicate item errors are expected and should not cause the operation to fail.
* PARTIAL – Add items that are not duplicates.


In [33]:
import com.aerospike.client.cdt.ListPolicy;
import com.aerospike.client.cdt.ListWriteFlags;

int listUniqueFlags = ListWriteFlags.ADD_UNIQUE 
                        | ListWriteFlags.NO_FAIL 
                        | ListWriteFlags.PARTIAL;

ListPolicy policyOrderedNoDupes = new ListPolicy(ListOrder.ORDERED, listUniqueFlags); 
                                                 
System.out.println("Created List Policy for Ordering and Adding Only Unique Items.");

Created List Policy for Ordering and Adding Only Unique Items.


#### Append the Second Data Set Using the New Policy

In [34]:
Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
    ListOperation.appendItems(policyOrderedNoDupes, listLeadersBin, wr100m2List)
    );
Record fullLeadersList = client.get(null, key);


System.out.println("The complete, deduplicated Leaders List is: " 
                    + fullLeadersList.getValue(listLeadersBin));

The complete, deduplicated Leaders List is: [[9.58, Usain Bolt, Berlin, Germany, August 16, 2009], [9.69, Usain Bolt, Beijing, China, August 16, 2008], [9.72, Usain Bolt, New York, USA, May 31, 2008], [9.74, Asafa Powell, Rieti, Italy, September 9, 2007], [9.77, Asafa Powell, Athens, Greece, June 14, 2005], [9.77, Asafa Powell, Gateshead, England, June 11, 2006], [9.77, Asafa Powell, Zurich, Switzerland, August 18, 2006], [9.79, Maurice Greene, Athens, Greece, June 16, 1999], [9.84, Donovan Bailey, Atlanta, USA, July 27, 1996], [9.85, Leroy Burrell, Lausanne, Switzerland, July 6, 1994], [9.86, Carl Lewis, Tokyo, Japan, August 25, 1991], [9.9, Leroy Burrell, New York, USA, June 14, 1991], [9.92, Carl Lewis, Seoul, South Korea, September 24, 1988], [9.93, Calvin Smith, Colorado Springs, USA, July 3, 1983], [9.93, Carl Lewis, Rome, Italy, August 30, 1987], [9.95, Jim Hines, Mexico City, Mexico, October 14, 1968], [10.02, Charles Greene, Mexico City, Mexico, October 13, 1968], [10.03, Jim 

# DELETING the Records and Closing Server Connection

## Delete the Record from Aerospike

Use the **asinfo** administration tool to drop the index containing our list data.

In [35]:
%sh asinfo -v "truncate:namespace=test;set=listset1;"
System.out.println("Record deleted.");

Record deleted.


## Close the Connection to Aerospike

In [36]:
client.close();
System.out.println("Server connection closed.");

Server connection closed.


# Code Summary

Here is a collection of all of the non-Jupyter code from this tutorial.

## Overview

1. Import Java Libraries.
2. Import Aerospike Client Libraries.
3. Start the Aerospike Client.
4. Compare Sorted and Unsorted Integer Lists.
    1. Create Two Identical Lists.
    2. Sort One of the Lists.
    3. Add Elements to Both Lists.
5. Apply Rank Operations on Sorted and Unsorted Mixed Data Type Lists.
    1. Create Two Identical Lists.
    2. Sort One of the Lists.
    3. Find the Range Between `NIL` and `10`.
    4. Find the Range Between `[ ]` and `INF`.
6. Apply Rank Operations on Nested Lists of Varying Length.
    1. Create Two Empty Lists.
    2. Sort One of the Lists.
    3. Put Lists of Varying Lengths in Both.
7. Apply Interval Operations on Nested Lists.
    1. Create a List of Tuples.
    2. Put the List of Tuples in an Ordered and an Unordered Bin.
    3. Find the Interval between `[1, NIL]` and `[2, NIL]`.
    4. Find the Interval between `[1, NIL]` and `[1, INF]`.
8. Example 1 – IoT Model.
    1. Create Sample Data.
    2. Order the List.
    3. Find the Temperature between Minute 14063 and 14064.
9. Example 2 – 100 Meter Leaderboard Model.
    1. Create Overlapping Data Sets.
    2. Put the First Data Set in a Sorted Aerospike Bin
    3. Find the Closest World Record to 10.0 Seconds.
    4. Insert the Overlapping Data Set.
10. Close the Client Connections.

In [37]:
// Import Java Libraries

import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;
import java.util.Map;


// Import Aerospike Client Libraries.

import com.aerospike.client.Key;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;
import com.aerospike.client.Record;
import com.aerospike.client.Operation;
import com.aerospike.client.Value;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListOrder;
import com.aerospike.client.cdt.ListReturnType;
import com.aerospike.client.cdt.ListPolicy;
import com.aerospike.client.cdt.ListWriteFlags;


// Start the Aerospike Client

AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");
System.out.println();

// Compare Sorted and Unsorted Integer Lists.
//    1. Create Two Identical Lists.
//    2. Sort One of the Lists.
//    3. Add Elements to Both Lists.

ArrayList<Integer> origListInt = new ArrayList<Integer>();
origListInt.add(1);
ArrayList<Value> addListInt = new ArrayList<Value>();
addListInt.add(Value.get(4));
addListInt.add(Value.get(7));
addListInt.add(Value.get(3));
addListInt.add(Value.get(9));
addListInt.add(Value.get(26));
addListInt.add(Value.get(11));

Integer theKey = 0;
String listSet = "orderset1";
String listNamespace = "test";
String listOrderedBin = "ordered";
String listUnorderedBin = "unordered";
ClientPolicy clientPolicy = new ClientPolicy();



Key key = new Key(listNamespace, listSet, theKey);
Bin bin1 = new Bin(listOrderedBin, origListInt);
Bin bin2 = new Bin(listUnorderedBin, origListInt);


client.put(clientPolicy.writePolicyDefault, key, bin1, bin2);
Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
                        ListOperation.setOrder(listOrderedBin, ListOrder.ORDERED),
                        ListOperation.appendItems(listOrderedBin, addListInt),
                        ListOperation.appendItems(listUnorderedBin, addListInt) 
                        );                       
Record finalListComp = client.get(null, key);

System.out.println("Put list into record with bins: " + listOrderedBin + " and " 
                    + listUnorderedBin);
System.out.println("Original Integer List: " + origListInt);
System.out.println("Appending the following list items to both lists: " + addListInt);
System.out.println("The Unordered List is " + finalListComp.getValue(listUnorderedBin));
System.out.println("The Ordered List is " + finalListComp.getValue(listOrderedBin));
System.out.println();

// Apply Rank Operations on Sorted and Unsorted Mixed Data Type Lists
//    1. Create Two Identical Lists.
//    2. Sort One of the Lists.
//    3. Find the Range Between NIL and 10.
//    4. Find the Range Between [ ] and INF.

ArrayList<Value> subMixedList = new ArrayList<Value>();
subMixedList.add(Value.get(25));
subMixedList.add(Value.get("y"));
subMixedList.add(Value.get(1));

System.out.println("Sub Mixed Type List: " + subMixedList);

ArrayList<Value> mixedDataList = new ArrayList<Value>();
mixedDataList.add(Value.get("c"));
mixedDataList.add(Value.get(3));
mixedDataList.add(Value.get(26));
mixedDataList.add(Value.get(subMixedList));
mixedDataList.add(Value.get(11));
mixedDataList.add(Value.get(9));
mixedDataList.add(Value.get(4));
mixedDataList.add(Value.get("a"));
mixedDataList.add(Value.get(1));
mixedDataList.add(Value.get(7));

Value lowEnd0 = Value.NULL;
Value highEnd0 = Value.get(10);

ArrayList<Value> emptyList = new ArrayList<Value>();

Value lowEnd1 = Value.get("a");
Value highEnd1 = Value.get(emptyList);

Value lowEnd2 = Value.get(emptyList);
Value highEnd2 = Value.INFINITY;


Integer mixedKey = 1;
String listSet = "orderset1";

Key key = new Key(listNamespace, listSet, mixedKey);
Bin bin1 = new Bin(listOrderedBin, mixedDataList);
Bin bin2 = new Bin(listUnorderedBin, mixedDataList);

client.put(clientPolicy.writePolicyDefault, key, bin1, bin2);

// Note – The following can be combined into one atomic operate.
Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
                ListOperation.setOrder(listOrderedBin, ListOrder.ORDERED),
                ListOperation.getByValueRange(listOrderedBin, lowEnd0, highEnd0, 
                                              ListReturnType.VALUE),
                ListOperation.getByValueRange(listUnorderedBin, lowEnd0, highEnd0, 
                                              ListReturnType.VALUE)
                );
                
Record rangeANullList = client.get(null, key);
rangeANullList = client.operate(client.writePolicyDefault, key, 
                ListOperation.getByValueRange(listOrderedBin, lowEnd1, highEnd1, 
                                              ListReturnType.VALUE),
                ListOperation.getByValueRange(listUnorderedBin, lowEnd1, highEnd1, 
                                              ListReturnType.VALUE)
                );

Record rangeNullInf = client.get(null, key);
rangeNullInf = client.operate(client.writePolicyDefault, key, 
                ListOperation.getByValueRange(listOrderedBin, lowEnd2, highEnd2, 
                                              ListReturnType.VALUE),
                ListOperation.getByValueRange(listUnorderedBin, lowEnd2, highEnd2, 
                                              ListReturnType.VALUE)
                );

System.out.println("Mixed Type Data List: " + mixedDataList);
System.out.println("Created record with bins: " + listOrderedBin + " and " 
                    + listUnorderedBin);
System.out.println("The Unordered List range between " + lowEnd0 + " and " + highEnd0 
    + " is " + record.getValue(listUnorderedBin));
System.out.println("The Ordered List range between " + lowEnd0 + " and " + highEnd0 
    + " is " + record.getValue(listOrderedBin));
System.out.println("The Unordered List range between " + lowEnd1 + " and " + highEnd1 
    + " is " + rangeANullList.getValue(listUnorderedBin));
System.out.println("The Ordered List range  between " + lowEnd1 + " and " + highEnd1 
    + " is " + rangeANullList.getValue(listOrderedBin));
System.out.println("The Unordered List range between " + lowEnd2 + " and " + highEnd2 
    + " is " + rangeNullInf.getValue(listUnorderedBin));
System.out.println("The Ordered List range  between " + lowEnd2 + " and " + highEnd2 
    + " is " + rangeNullInf.getValue(listOrderedBin));
System.out.println();    

// Apply Rank Operations on Nested Lists of Varying Length.
//    1. Create Two Empty Lists.
//    2. Sort One of the Lists.
//    3. Put Lists of Varying Lengths in Both.

ArrayList<Integer> thisTuple0 = new ArrayList<Integer>();
thisTuple0.add(1);
thisTuple0.add(1);
 
ArrayList<Integer> thisTuple1 = new ArrayList<Integer>();
thisTuple1.add(1);
thisTuple1.add(3);

ArrayList<Integer> thisTuple2 = new ArrayList<Integer>();
thisTuple2.add(1);
thisTuple2.add(2);
thisTuple2.add(0);
thisTuple2.add(1);

ArrayList<Integer> thisTuple3 = new ArrayList<Integer>();
thisTuple3.add(1);
thisTuple3.add(2);


Integer listOfListsKey = 2;
String listSet = "orderset1";

Key key = new Key(listNamespace, listSet, listOfListsKey);
Bin bin1 = new Bin(listOrderedBin, emptyList);
Bin bin2 = new Bin(listUnorderedBin, emptyList);

client.put(clientPolicy.writePolicyDefault, key, bin1, bin2);
                    
Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
    ListOperation.setOrder(listOrderedBin, ListOrder.ORDERED),
    ListOperation.append(listOrderedBin, Value.get(thisTuple0)),
    ListOperation.append(listUnorderedBin, Value.get(thisTuple0)),    
    ListOperation.append(listOrderedBin, Value.get(thisTuple1)),
    ListOperation.append(listUnorderedBin, Value.get(thisTuple1)),    
    ListOperation.append(listOrderedBin, Value.get(thisTuple2)),
    ListOperation.append(listUnorderedBin, Value.get(thisTuple2)),    
    ListOperation.append(listOrderedBin, Value.get(thisTuple3)),
    ListOperation.append(listUnorderedBin, Value.get(thisTuple3))    
    );
Record listWithTuples = client.get(null, key);

System.out.println("Created record with bins: " + listOrderedBin + " and " 
                    + listUnorderedBin);
System.out.println("Configured ordering policy to bin: " + listOrderedBin);
System.out.println("The Unordered List is " + listWithTuples.getValue(listUnorderedBin));
System.out.println("The Ordered List is " + listWithTuples.getValue(listOrderedBin));
System.out.println(); 

// Apply Interval Operations on Nested Lists.
//    1. Create a List of Tuples.
//    2. Put the List of Tuples in an Ordered and an Unordered Bin.
//    3. Find the Interval between [1, NIL] and [2, NIL].
//    4. Find the Interval between [1, NIL] and [1, INF].

ArrayList<Integer> subTuple0 = new ArrayList<Integer>();
subTuple0.add(2);
subTuple0.add(7);

ArrayList<Integer> subTuple1 = new ArrayList<Integer>();
subTuple1.add(3);
subTuple1.add(1);

ArrayList<Integer> subTuple2 = new ArrayList<Integer>();
subTuple2.add(1);
subTuple2.add(2);

ArrayList<Integer> subTuple3 = new ArrayList<Integer>();
subTuple3.add(2);
subTuple3.add(5);

ArrayList<Integer> subTuple4 = new ArrayList<Integer>();
subTuple4.add(1);
subTuple4.add(1);

ArrayList<Integer> subTuple5 = new ArrayList<Integer>();
subTuple5.add(0);
subTuple5.add(3);

ArrayList<Value> tupleList = new ArrayList<Value>();
tupleList.add(Value.get(subTuple0));
tupleList.add(Value.get(subTuple1));
tupleList.add(Value.get(subTuple2));
tupleList.add(Value.get(subTuple3));
tupleList.add(Value.get(subTuple4));
tupleList.add(Value.get(subTuple5));


ArrayList<Value> lowTuple0 = new ArrayList<Value>();
lowTuple0.add(Value.get(1));
lowTuple0.add(Value.NULL);

ArrayList<Value> highTuple0 = new ArrayList<Value>();
highTuple0.add(Value.get(2));
highTuple0.add(Value.NULL);

ArrayList<Value> lowTuple1 = new ArrayList<Value>();
lowTuple1.add(Value.get(1));
lowTuple1.add(Value.NULL);

ArrayList<Value> highTuple1 = new ArrayList<Value>();
highTuple1.add(Value.get(1));
highTuple1.add(Value.INFINITY);


Integer tupleKey = 3;
String listSet = "orderset1";

Key key = new Key(listNamespace, listSet, tupleKey);
Bin bin1 = new Bin(listOrderedBin, tupleList);
Bin bin2 = new Bin(listUnorderedBin, tupleList);

client.put(clientPolicy.writePolicyDefault, key, bin1, bin2);

Record record = client.get(null, key);
record = client.operate(client.writePolicyDefault, key, 
    ListOperation.setOrder(listOrderedBin, ListOrder.ORDERED)
    );
Record between1and2 = client.get(null, key);
between1and2 = client.operate(client.writePolicyDefault, key, 
    ListOperation.getByValueRange(listOrderedBin, Value.get(lowTuple0), Value.get(highTuple0), 
        ListReturnType.VALUE),
    ListOperation.getByValueRange(listUnorderedBin, Value.get(lowTuple0), Value.get(highTuple0), 
        ListReturnType.VALUE)    
    );
Record between1NILand1INF = client.get(null, key);
between1NILand1INF = client.operate(client.writePolicyDefault, key, 
    ListOperation.getByValueRange(listOrderedBin, Value.get(lowTuple1), Value.get(highTuple1), 
        ListReturnType.VALUE),
    ListOperation.getByValueRange(listUnorderedBin, Value.get(lowTuple1), Value.get(highTuple1), 
        ListReturnType.VALUE)    
    );

System.out.println("Tuple List: " + tupleList);
System.out.println("Created record with bins: " + listOrderedBin + " and " 
                    + listUnorderedBin);
System.out.println("The Unordered List between " + lowTuple0 + " and " + highTuple0 + " is " 
    + between1and2.getValue(listUnorderedBin));
System.out.println("The Ordered List between " + lowTuple0 + " and " + highTuple0 + " is " 
    + between1and2.getValue(listOrderedBin));
System.out.println("The Unordered List between " + lowTuple1 + " and " + highTuple1 + " is " 
    + between1NILand1INF.getValue(listUnorderedBin));
System.out.println("The Ordered List between " + lowTuple1 + " and " + highTuple1 + " is " 
    + between1NILand1INF.getValue(listOrderedBin));
System.out.println();


// Example 1 – IoT Model.
//    1. Create Sample Data.
//    2. Order the List.
//    3. Find the Temperature between Minute 14063 and 14064.

ArrayList<Value> subTuple0 = new ArrayList<Value>();
subTuple0.add(Value.get(14062));
subTuple0.add(Value.get(39.04));

ArrayList<Value> subTuple1 = new ArrayList<Value>();
subTuple1.add(Value.get(14063));
subTuple1.add(Value.get(39.78));

ArrayList<Value> subTuple2 = new ArrayList<Value>();
subTuple2.add(Value.get(14065));
subTuple2.add(Value.get(40.07));

ArrayList<Value> subTuple3 = new ArrayList<Value>();
subTuple3.add(Value.get(14064));
subTuple3.add(Value.get(40.89));

ArrayList<Value> subTuple4 = new ArrayList<Value>();
subTuple4.add(Value.get(14066));
subTuple4.add(Value.get(41.18));

ArrayList<Value> subTuple5 = new ArrayList<Value>();
subTuple5.add(Value.get(14067));
subTuple5.add(Value.get(40.93));

ArrayList<Value> tupleListIoT = new ArrayList<Value>();
tupleListIoT.add(Value.get(subTuple0));
tupleListIoT.add(Value.get(subTuple1));
tupleListIoT.add(Value.get(subTuple2));
tupleListIoT.add(Value.get(subTuple3));
tupleListIoT.add(Value.get(subTuple4));
tupleListIoT.add(Value.get(subTuple5));

ArrayList<Value> lowTuple = new ArrayList<Value>();
lowTuple.add(Value.get(14063));
lowTuple.add(Value.NULL);

ArrayList<Value> highTuple = new ArrayList<Value>();
highTuple.add(Value.get(14064));
highTuple.add(Value.INFINITY);


Integer keyIoT = 4;
String listSet = "orderset1";
ClientPolicy clientPolicy = new ClientPolicy();

Key key = new Key(listNamespace, listSet, keyIoT);
Bin bin1 = new Bin(listUnorderedBin, tupleListIoT);

client.put(clientPolicy.writePolicyDefault, key, bin1);
Record between14063and14064 = client.get(null, key);
between14063and14064 = client.operate(client.writePolicyDefault, key, 
    ListOperation.getByValueRange(listUnorderedBin, Value.get(lowTuple), Value.get(highTuple), 
        ListReturnType.VALUE)    
    );

System.out.println("IoT Tuple List: " + tupleListIoT);
System.out.println("Created record with bin: " + listUnorderedBin);
System.out.println("The readings between " + lowTuple + " and " + highTuple + " are " 
    + between14063and14064.getValue(listUnorderedBin));
System.out.println();


// Example 2 – 100 Meter Leaderboard Model.
//    1. Create Overlapping Data Sets.
//    2. Put the First Data Set in a Sorted Aerospike Bin
//    3. Find the Closest World Record to 10.0 Seconds.
//    4. Insert the Overlapping Data Set.

ArrayList<Value> wr100One0 = new ArrayList<Value>();
wr100One0.add(Value.get(10.06));
wr100One0.add(Value.get("Bob Hayes"));
wr100One0.add(Value.get("Tokyo, Japan"));
wr100One0.add(Value.get("October 15, 1964"));

ArrayList<Value> wr100One1 = new ArrayList<Value>();
wr100One1.add(Value.get(10.03));
wr100One1.add(Value.get("Jim Hines"));
wr100One1.add(Value.get("Sacramento, USA"));
wr100One1.add(Value.get("June 20,1968"));

ArrayList<Value> wr100One2 = new ArrayList<Value>();
wr100One2.add(Value.get(10.02));
wr100One2.add(Value.get("Charles Greene"));
wr100One2.add(Value.get("Mexico City, Mexico"));
wr100One2.add(Value.get("October 13, 1968"));

ArrayList<Value> wr100One3 = new ArrayList<Value>();
wr100One3.add(Value.get(9.95));
wr100One3.add(Value.get("Jim Hines"));
wr100One3.add(Value.get("Mexico City, Mexico"));
wr100One3.add(Value.get("October 14, 1968"));

ArrayList<Value> wr100One4 = new ArrayList<Value>();
wr100One4.add(Value.get(9.93));
wr100One4.add(Value.get("Calvin Smith"));
wr100One4.add(Value.get("Colorado Springs, USA"));
wr100One4.add(Value.get("July 3, 1983"));

ArrayList<Value> wr100One5 = new ArrayList<Value>();
wr100One5.add(Value.get(9.93));
wr100One5.add(Value.get("Carl Lewis"));
wr100One5.add(Value.get("Rome, Italy"));
wr100One5.add(Value.get("August 30, 1987"));

ArrayList<Value> wr100One6 = new ArrayList<Value>();
wr100One6.add(Value.get(9.92));
wr100One6.add(Value.get("Carl Lewis"));
wr100One6.add(Value.get("Seoul, South Korea"));
wr100One6.add(Value.get("September 24, 1988"));


ArrayList<Value> wr100m1List = new ArrayList<Value>();
wr100m1List.add(Value.get(wr100One0));
wr100m1List.add(Value.get(wr100One1));
wr100m1List.add(Value.get(wr100One2));
wr100m1List.add(Value.get(wr100One3));
wr100m1List.add(Value.get(wr100One4));
wr100m1List.add(Value.get(wr100One5));
wr100m1List.add(Value.get(wr100One6));

ArrayList<Value> wr100Two0 = new ArrayList<Value>();
wr100Two0.add(Value.get(9.93));
wr100Two0.add(Value.get("Carl Lewis"));
wr100Two0.add(Value.get("Rome, Italy"));
wr100Two0.add(Value.get("August 30, 1987"));

ArrayList<Value> wr100Two1 = new ArrayList<Value>();
wr100Two1.add(Value.get(9.92));
wr100Two1.add(Value.get("Carl Lewis"));
wr100Two1.add(Value.get("Seoul, South Korea"));
wr100Two1.add(Value.get("September 24, 1988"));

ArrayList<Value> wr100Two2 = new ArrayList<Value>();
wr100Two2.add(Value.get(9.90));
wr100Two2.add(Value.get("Leroy Burrell"));
wr100Two2.add(Value.get("New York, USA"));
wr100Two2.add(Value.get("June 14, 1991"));

ArrayList<Value> wr100Two3 = new ArrayList<Value>();
wr100Two3.add(Value.get(9.86));
wr100Two3.add(Value.get("Carl Lewis"));
wr100Two3.add(Value.get("Tokyo, Japan"));
wr100Two3.add(Value.get("August 25, 1991"));

ArrayList<Value> wr100Two4 = new ArrayList<Value>();
wr100Two4.add(Value.get(9.85));
wr100Two4.add(Value.get("Leroy Burrell"));
wr100Two4.add(Value.get("Lausanne, Switzerland"));
wr100Two4.add(Value.get("July 6, 1994"));

ArrayList<Value> wr100Two5 = new ArrayList<Value>();
wr100Two5.add(Value.get(9.84));
wr100Two5.add(Value.get("Donovan Bailey"));
wr100Two5.add(Value.get("Atlanta, USA"));
wr100Two5.add(Value.get("July 27, 1996"));

ArrayList<Value> wr100Two6 = new ArrayList<Value>();
wr100Two6.add(Value.get(9.79));
wr100Two6.add(Value.get("Maurice Greene"));
wr100Two6.add(Value.get("Athens, Greece"));
wr100Two6.add(Value.get("June 16, 1999"));

ArrayList<Value> wr100Two7 = new ArrayList<Value>();
wr100Two7.add(Value.get(9.77));
wr100Two7.add(Value.get("Asafa Powell"));
wr100Two7.add(Value.get("Athens, Greece"));
wr100Two7.add(Value.get("June 14, 2005"));

ArrayList<Value> wr100Two8 = new ArrayList<Value>();
wr100Two8.add(Value.get(9.77));
wr100Two8.add(Value.get("Asafa Powell"));
wr100Two8.add(Value.get("Gateshead, England"));
wr100Two8.add(Value.get("June 11, 2006"));

ArrayList<Value> wr100Two9 = new ArrayList<Value>();
wr100Two9.add(Value.get(9.77));
wr100Two9.add(Value.get("Asafa Powell"));
wr100Two9.add(Value.get("Zurich, Switzerland"));
wr100Two9.add(Value.get("August 18, 2006"));

ArrayList<Value> wr100Two10 = new ArrayList<Value>();
wr100Two10.add(Value.get(9.74));
wr100Two10.add(Value.get("Asafa Powell"));
wr100Two10.add(Value.get("Rieti, Italy"));
wr100Two10.add(Value.get("September 9, 2007"));

ArrayList<Value> wr100Two11 = new ArrayList<Value>();
wr100Two11.add(Value.get(9.72));
wr100Two11.add(Value.get("Usain Bolt"));
wr100Two11.add(Value.get("New York, USA"));
wr100Two11.add(Value.get("May 31, 2008"));

ArrayList<Value> wr100Two12 = new ArrayList<Value>();
wr100Two12.add(Value.get(9.69));
wr100Two12.add(Value.get("Usain Bolt"));
wr100Two12.add(Value.get("Beijing, China"));
wr100Two12.add(Value.get("August 16, 2008"));

ArrayList<Value> wr100Two13 = new ArrayList<Value>();
wr100Two13.add(Value.get(9.58));
wr100Two13.add(Value.get("Usain Bolt"));
wr100Two13.add(Value.get("Berlin, Germany"));
wr100Two13.add(Value.get("August 16, 2009"));


ArrayList<Value> wr100m2List = new ArrayList<Value>();
wr100m2List.add(Value.get(wr100Two0));
wr100m2List.add(Value.get(wr100Two1));
wr100m2List.add(Value.get(wr100Two2));
wr100m2List.add(Value.get(wr100Two3));
wr100m2List.add(Value.get(wr100Two4));
wr100m2List.add(Value.get(wr100Two5));
wr100m2List.add(Value.get(wr100Two6));
wr100m2List.add(Value.get(wr100Two7));
wr100m2List.add(Value.get(wr100Two8));
wr100m2List.add(Value.get(wr100Two9));
wr100m2List.add(Value.get(wr100Two10));
wr100m2List.add(Value.get(wr100Two11));
wr100m2List.add(Value.get(wr100Two12));
wr100m2List.add(Value.get(wr100Two13));

ArrayList<Value> searchTuple = new ArrayList<Value>();
searchTuple.add(Value.get(10.0));
searchTuple.add(Value.NULL);

Integer searchRank = -1;
Integer searchQty = 2;


Integer keyWR = 5;
String listSet = "orderset1";
String listLeadersBin = "leaders";
int listUniqueFlags = ListWriteFlags.ADD_UNIQUE 
                        | ListWriteFlags.NO_FAIL 
                        | ListWriteFlags.PARTIAL;

Key key = new Key(listNamespace, listSet, keyWR);
Bin bin1 = new Bin(listLeadersBin, wr100m1List);
ListPolicy policyOrderedNoDupes = new ListPolicy(ListOrder.ORDERED, listUniqueFlags); 



client.put(clientPolicy.writePolicyDefault, key, bin1);
Record closestTo10 = client.get(null, key);
closestTo10 = client.operate(client.writePolicyDefault, key, 
    ListOperation.setOrder(listLeadersBin, ListOrder.ORDERED),
    ListOperation.getByValueRelativeRankRange(listLeadersBin, Value.get(searchTuple), 
        searchRank, searchQty, ListReturnType.VALUE)
    );
Record partialLeaderList = client.get(null, key);
partialLeaderList = client.operate(client.writePolicyDefault, key, 
    ListOperation.appendItems(policyOrderedNoDupes, listLeadersBin, wr100m2List)
    );
Record fullLeadersList = client.get(null, key);


System.out.println("100m World Record Holder List 1: "); 
System.out.println(wr100m1List);
System.out.println();
System.out.println("100m World Record Holder List 2: ");
System.out.println(wr100m2List);
System.out.println();
System.out.println("Created Record with Ordered Bin: " + listLeadersBin);
System.out.println();
System.out.println("The Closest World Records to 10.0 seconds are " + 
    closestTo10.getValue(listLeadersBin));
System.out.println();
System.out.println("Created List Policy for Ordering and Adding Only Unique Items.");
System.out.println();
System.out.println("The complete, deduplicated Leaders List is: "); 
System.out.println(fullLeadersList.getValue(listLeadersBin));
System.out.println();

// Close the Client Connections.
client.close();
System.out.println("Server connection closed.");




Initialized the client and connected to the cluster.

Put list into record with bins: ordered and unordered
Original Integer List: [1]
Appending the following list items to both lists: [4, 7, 3, 9, 26, 11]
The Unordered List is [1, 4, 7, 3, 9, 26, 11]
The Ordered List is [1, 3, 4, 7, 9, 11, 26]

Sub Mixed Type List: [25, y, 1]
Mixed Type Data List: [c, 3, 26, [25, y, 1], 11, 9, 4, a, 1, 7]
Created record with bins: ordered and unordered
The Unordered List range between null and 10 is [3, 9, 4, 1, 7]
The Ordered List range between null and 10 is [1, 3, 4, 7, 9]
The Unordered List range between a and [] is [c, a]
The Ordered List range  between a and [] is [a, c]
The Unordered List range between [] and INF is [[25, y, 1]]
The Ordered List range  between [] and INF is [[25, y, 1]]

Created record with bins: ordered and unordered
Configured ordering policy to bin: ordered
The Unordered List is [[1, 1], [1, 3], [1, 2, 0, 1], [1, 2]]
The Ordered List is [[1, 1], [1, 2], [1, 2, 0, 1], [1, 3]]

# Takeaway – Sort for App Read/Write Performance

Since, the results are the same whether the data is ordered or unordered in the database. The question becomes, "Does sorting the data benefit application performance for reads/writes?"

# What's Next? 



## Next Steps

Have questions? Don't hesitate to reach out if you have additional questions about working with lists at https://discuss.aerospike.com/.

Want to check out other Java notebooks?
1. [Hello, World](hello_world.ipynb)
2. [Aerospike Query and UDF](query_udf.ipynb)
3. [Simple Put Get Example](SimplePutGetExample.ipynb)
4. [Working with Twitter Data](tweetaspike.ipynb)
5. [Working with Lists](java-working_with_lists.ipynb)

Are you running this from Binder? [Download the Aerospike Notebook Repo](https://github.com/aerospike/aerospike-dev-notebooks.docker) and work with Aerospike Database and Jupyter locally using a Docker container.

## Additional Resources

* What else can we do in Java? See the [Aerospike Java Client](https://github.com/aerospike/aerospike-client-java). 
* What other ways can we work with Lists? Take a look at [Aerospike's List Operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/ListOperation.html).
* What are Namespaces, Sets, and Bins? Check out the [Aerospike Data Model](https://www.aerospike.com/docs/architecture/data-model.html). 
* How robust is the Aerospike database? Browse the [Aerospike Database Architecture](https://www.aerospike.com/docs/architecture/index.html).